# Example

+ [1. Convert dtypes](#1.-Convert-dtypes)
+ [2. Altering columns](#2.-Altering-columns)
+ [3. Checks](#3.-Checks)
+ [4. Imported](#4.-Imported)
+ [5. Formed tables](#5.-Formed-tables)

Need sections for other specific features to highlight

## Setup

Imports and setting options

In [1]:
import IPython.core.display as ICD
from datetime import datetime
import os
import glob
import logging
import pickle
import importlib

import pandas as pd
import numpy as np

from data_curation import DataCuration, Checks

pd.set_option('display.max_rows', 12)
pd.set_option('display.max_columns', None)

logger = logging.getLogger()

# 1. Convert dtypes

## Initialise the class

In [2]:
var_start_time = datetime.now()
dc_convert = DataCuration(var_start_time, '1')

## Get the list of files

In [3]:
def find_files(path):
    list_files = [os.path.abspath(file) for file in glob.glob(f'{path}/*_convert*.tsv')]
    return list_files


dc_convert.find_files(function=lambda path: find_files(path), path='data')

dc_convert.list_files

['C:\\Users\\georg\\Documents\\workspace\\modules\\data_curation\\examples\\data\\df_convert.tsv',
 'C:\\Users\\georg\\Documents\\workspace\\modules\\data_curation\\examples\\data\\df_convert_issues.tsv']

## Read in the data

In [4]:
def read_in(list_files):
    dict_tables = dict()
    for file in list_files:
        var_filename = file.split('\\')[-1]
        dict_tables[var_filename] = pd.read_csv(file, sep='\t', dtype=str)
    return dict_tables


dc_convert.reading_in(function=read_in)

dc_convert.tables

{'df_convert.tsv':   object int float        date
 0      A   1   0.6  2019-01-01
 1      B   4   5.2  2019-02-05
 2      C   1   5.6  2018-12-17
 3      D  10  15.9  2019-07-18
 4      E  -8   4.7  2018-03-09,
 'df_convert_issues.tsv':   object  int float        date
 0      A    1   0.6  2019-02-29
 1      B  4.5     A  2019-22-05
 2      C    1   5.6  2018-12-17
 3      D    b  15.9  2019-09-31
 4      5   -8   4.7  2018-03-09}

## Convert the information

In [5]:
def func_try_float_cnv(x): 
    try:
        var = float(x)
    except:
        return True
    return False


def func_try_int_cnv(x): 
    try:
        var = int(x)
    except:
        return True
    return False


def func_str_cnv(s):
    var_is_null_pre = s.isnull().sum()
    s_cnv = s.map(func_to_int).str.strip()
    var_is_null_post = s_cnv.isnull().sum()
    if var_is_null_post != var_is_null_pre:
        raise ValueError
    return s_cnv


def func_to_int(x):
    try:
        return int(x)
    except: 
        return x


def func_try_str_cnv(s):
    var_is_null_pre = s.isnull().sum()
    s_cnv = s.map(func_to_int).str.strip()
    var_is_null_post = s_cnv.isnull().sum()
    return s != s_cnv


def func_try_date_cnv(x):
    if pd.isnull(x):
        return False
    if pd.isnull(pd.to_datetime(x, format='%Y-%m-%d', errors='coerce')):
        return True
    return False


dict_convert = {
    'float': {
        'columns': ['float'],
        'dtypes': ['float'],
        'functions': {
            1: lambda df, col, **kwargs: df[col].astype(float)
        },
        'idx_function': lambda df, col, **kwargs: df[col].map(func_try_float_cnv)
    },
    'int': {
        'columns': ['int'],
        'dtypes': ['int'],
        'functions': {
            1: lambda df, col, **kwargs: df[col].astype(int)
        },
        'idx_function': lambda df, col, **kwargs: df[col].map(func_try_int_cnv)
    },
    'object': {
        'columns': ['object'],
        'dtypes': [],
        'functions': {
            1: lambda df, col, **kwargs: func_str_cnv(df[col])
        },
        'idx_function': lambda df, col, **kwargs: func_try_str_cnv(df[col])
    },
    'date': {
        'columns': ['date'],
        'dtypes': ['date', '[ns]'],
        'functions': {
            1: lambda df, col, **kwargs: pd.to_datetime(df[col], format='%Y-%m-%d')
        },
        'idx_function': lambda df, col, **kwargs: df[col].map(func_try_date_cnv)
    }
}

In [6]:
dc_convert.convert_columns(dictionary=dict_convert)

The conversion failed for keys float, 1, trying next
The conversion for column float for convert_key float failed.
The conversion failed for keys int, 1, trying next
The conversion for column int for convert_key int failed.
The conversion failed for keys object, 1, trying next
The conversion for column object for convert_key object failed.
The conversion failed for keys date, 1, trying next
The conversion for column date for convert_key date failed.


In [7]:
dc_convert.df_issues

,key_1,key_2,key_3,file,sub_file,step_number,category,issue_short_desc,issue_long_desc,column,issue_count,issue_idx,grouping
0,1,None,None,df_convert_issues.tsv,NaN,0,NaN,,The conversion failed to format float,float,1,1,2019-11-24 14:12:43.461904
1,1,None,None,df_convert_issues.tsv,NaN,0,NaN,,The conversion failed to format int,int,2,"1, 3",2019-11-24 14:12:43.461904
2,1,None,None,df_convert_issues.tsv,NaN,0,NaN,,The conversion failed to format object,object,1,4,2019-11-24 14:12:43.461904
3,1,None,None,df_convert_issues.tsv,NaN,0,NaN,,The conversion failed to format date,date,3,"0, 1, 3",2019-11-24 14:12:43.461904


In [8]:
dc_convert.tables['df_convert_issues.tsv']

,object,int,float,date
0,A,1,0.6,2019-02-29
1,B,4.5,A,2019-22-05
2,C,1,5.6,2018-12-17
3,D,b,15.9,2019-09-31
4,5,-8,4.7,2018-03-09


# 2. Altering columns

Disable the logging messages

In [9]:
logger.setLevel(logging.CRITICAL)
# logger.setLevel(logging.INFO)  # To enable them again

## Initialise the class

In [10]:
var_start_time = datetime.now()
dc_alter = DataCuration(var_start_time, '2')

## Set the data

In [11]:
dc_alter.set_table(
    {
        'df_alterations.tsv': pd.DataFrame(
            [
                ('A', 2, 'key_1'),
                ('B', 199, 'key_2'),
                ('C', -1, 'key_1'),
                ('D', 20, 'key_3'),
                ('E', 6, 'key_2')
            ],
            columns=['to_map', 'add_1', 'merge_key']
        ),
        'df_alterations_issues.tsv': pd.DataFrame(
            [
                ('A', 2, 'key_1'),
                ('B', 199, 2),
                ('C', -1, 'key_1'),
                (['D'], 'a', 'key_3'),
                ('E', 6, 'key_2')
            ],
            columns=['to_map', 'add_1', 'merge_key']
        )
    }
)

In [12]:
dc_alter.tables

{'df_alterations.tsv':   to_map  add_1 merge_key
 0      A      2     key_1
 1      B    199     key_2
 2      C     -1     key_1
 3      D     20     key_3
 4      E      6     key_2,
 'df_alterations_issues.tsv':   to_map add_1 merge_key
 0      A     2     key_1
 1      B   199         2
 2      C    -1     key_1
 3    [D]     a     key_3
 4      E     6     key_2}

## Alter the information

In [13]:
df_mapping = pd.DataFrame(
    [
        ('key_1', 1),
        ('key_2', 2),
        ('key_3', 3)
    ], 
    columns=['merge_key', 'out_value']
)


def func_alter_merge(df, df_mapping):
    df_mapped = pd.merge(
        df,
        df_mapping,
        on='merge_key',
        how='left'
    )
    if (
        df_mapped['out_value'].isnull().sum() != 
        df['merge_key'].isnull().sum()
    ):
        raise ValueError
    return df_mapped


dict_alter = {
    '01': {
        'type': 'new_col',
        'col_name': 'key',
        'function': lambda df, keys, **kwargs: keys[0]
    },
    '02': {
        'type': 'new_col',
        'col_name': 'done_add_1',
        'function': lambda df, keys, **kwargs: df['add_1'] + 1,
        'idx_function': lambda df, keys, **kwargs: 
            df['add_1'].map(
                lambda x: type(x).__name__).map(
                lambda x: ('int' in x) | ('float' in x)).map(
                {True: False, False: True})
    },
    '03': {
        'type': 'new_col',
        'col_name': 'mapped',
        'function': lambda df, keys, **kwargs: df['to_map'].map({
            'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5}),
        'idx_function': lambda df, keys, **kwargs: 
            ~df['to_map'].astype(str).isin(['A', 'B', 'C', 'D', 'E'])
    },
    '04': {
        'type': 'map_df',
        'function': lambda df, keys, **kwargs: 
            func_alter_merge(df, kwargs['df_mapping']),
        'idx_function': lambda df, keys, **kwargs: 
            ~df['merge_key'].isin(['key_1', 'key_2', 'key_3', np.nan])
    }
}

In [14]:
dc_alter.alter_tables(dictionary=dict_alter, df_mapping=df_mapping)

In [15]:
dc_alter.df_issues

,key_1,key_2,key_3,file,sub_file,step_number,category,issue_short_desc,issue_long_desc,column,issue_count,issue_idx,grouping
0,2,None,None,df_alterations_issues.tsv,NaN,0,NaN,,For type new_col the function for alter_key 02...,done_add_1,1,3,2019-11-24 14:12:43.790151
1,2,None,None,df_alterations_issues.tsv,NaN,0,NaN,,For type new_col the function for alter_key 03...,mapped,1,3,2019-11-24 14:12:43.790151
2,2,None,None,df_alterations_issues.tsv,NaN,0,NaN,,For type map_df the function for alter_key 04 ...,NaN,1,1,2019-11-24 14:12:43.790151


In [16]:
dc_alter.tables['df_alterations.tsv']

,to_map,add_1,merge_key,key,done_add_1,mapped,out_value
0,A,2,key_1,2,3,1,1
1,B,199,key_2,2,200,2,2
2,C,-1,key_1,2,0,3,1
3,D,20,key_3,2,21,4,3
4,E,6,key_2,2,7,5,2


In [17]:
dc_alter.tables['df_alterations_issues.tsv']

,to_map,add_1,merge_key,key
0,A,2,key_1,2
1,B,199,2,2
2,C,-1,key_1,2
3,[D],a,key_3,2
4,E,6,key_2,2


# 3. Checks

## Initialise the class

In [18]:
var_start_time = datetime.now()
ch_checks = Checks(var_start_time, '3')

## Set the data

In [19]:
dict_data = {
    'df_checks.pkl': pickle.load(open('data/df_checks.pkl', 'rb')),
    'df_checks_issues.pkl': pickle.load(open('data/df_checks_issues.pkl', 'rb'))
}

## Check the data

In [20]:
dict_cat_1_map = {
    'A': ['a', 'z'],
    'B': ['b'],
    'C': ['c'],
    'D': ['d'],
    'Y': ['y'],
    'Z': ['z']
}

dict_checks = {
    'Number should be greater than 0': {
        'calc_condition': lambda df, col, **kwargs: df['number'] <= 0
    },
    'Number should be greater than 2': {
        "columns": ['number'],
        'calc_condition': lambda df, col, **kwargs: df[col] <= 2,
        'category': 'severe'
    },
    'check values in list': {
        'columns': ['category_1'],
        'calc_condition': lambda df, col, **kwargs: ~df[col].isin(['A', 'B', 'C', 'D']),
        'long_description': lambda df, col, condition, **kwargs: 
            f"The invalid values are: {df.loc[~df[col].isin(['A', 'B', 'C', 'D'])][col].unique().tolist()}"
    },
    'The category_1 column can only map to certain values': {
        'calc_condition': lambda df, col, **kwargs: [
            item[1] not in dict_cat_1_map[item[0]] for item in 
            df[['category_1', 'category_2']].values.tolist()
        ],
        'check_condition': lambda df, col, condition, **kwargs: sum(condition) > 0,
        'count_condition': lambda df, col, condition, **kwargs: sum(condition),
        'index_position': lambda df, col, condition, **kwargs: pd.Series(condition),
        'relevant_columns': lambda df, col, condition, **kwargs: 'category_1, category_2',
        'long_description': lambda df, col, condition, **kwargs: (
            f"The values that have no mapping are: "
            f"{df.loc[pd.Series(condition)]['category_1'].unique().tolist()}"
        )
    }
}

In [21]:
ch_checks.apply_checks(dict_data, dictionary=dict_checks)

In [22]:
ch_checks.df_issues

,key_1,key_2,key_3,file,sub_file,step_number,category,issue_short_desc,issue_long_desc,column,issue_count,issue_idx,grouping
0,3,None,None,df_checks_issues.pkl,NaN,0,NaN,Number should be greater than 0,,NaN,1,4,2019-11-24 14:12:44.026966
1,3,None,None,df_checks_issues.pkl,NaN,0,severe,Number should be greater than 2,,number,2,"0, 4",2019-11-24 14:12:44.026966
2,3,None,None,df_checks_issues.pkl,NaN,0,NaN,check values in list,"The invalid values are: ['Z', 'Y']",category_1,2,"0, 2",2019-11-24 14:12:44.026966
3,3,None,None,df_checks_issues.pkl,NaN,0,NaN,The category_1 column can only map to certain ...,"The values that have no mapping are: ['Z', 'Y']","category_1, category_2",2,"0, 2",2019-11-24 14:12:44.026966


In [23]:
dict_data['df_checks_issues.pkl']

,number,category_1,category_2
0,1,Z,y
1,10,A,a
2,9,Y,b
3,4,B,b
4,-1,C,c


In [24]:
df_issues, df_issue = ch_checks.table_look(dict_data['df_checks_issues.pkl'], 0)
ICD.display(df_issues)
ICD.display(df_issue)

,key_1,key_2,key_3,file,sub_file,step_number,category,issue_short_desc,issue_long_desc,column,issue_count,issue_idx,grouping
0,3,None,None,df_checks_issues.pkl,NaN,0,NaN,Number should be greater than 0,,NaN,1,4,2019-11-24 14:12:44.026966


,number,category_1,category_2
4,-1,C,c


# 4. Imported

## Initialise the class

In [25]:
var_start_time = datetime.now()
ch_checks_imported = Checks(var_start_time, '4')

## Set the data

In [26]:
dict_data_imported = {
    'df_checks.pkl': pickle.load(open('data/df_checks.pkl', 'rb')),
    'df_checks_issues.pkl': pickle.load(open('data/df_checks_issues.pkl', 'rb'))
}

## Import the defined dictonary of checks

In [27]:
dict_checks = importlib.import_module('02_example').dict_checks
dict_checks.keys()

dict_keys(['Number should be greater than 0', 'Number should be greater than 2', 'check values in list', 'The category_1 column can only map to certain values'])

## Check the data

In [28]:
ch_checks_imported.apply_checks(dict_data, dictionary=dict_checks)

In [29]:
ch_checks_imported.df_issues

,key_1,key_2,key_3,file,sub_file,step_number,category,issue_short_desc,issue_long_desc,column,issue_count,issue_idx,grouping
0,4,None,None,df_checks_issues.pkl,NaN,0,NaN,Number should be greater than 0,,NaN,1,4,2019-11-24 14:12:44.269445
1,4,None,None,df_checks_issues.pkl,NaN,0,severe,Number should be greater than 2,,number,2,"0, 4",2019-11-24 14:12:44.269445
2,4,None,None,df_checks_issues.pkl,NaN,0,NaN,check values in list,"The invalid values are: ['Z', 'Y']",category_1,2,"0, 2",2019-11-24 14:12:44.269445
3,4,None,None,df_checks_issues.pkl,NaN,0,NaN,The category_1 column can only map to certain ...,"The values that have no mapping are: ['Z', 'Y']","category_1, category_2",2,"0, 2",2019-11-24 14:12:44.269445


In [30]:
dict_data_imported['df_checks_issues.pkl']

,number,category_1,category_2
0,1,Z,y
1,10,A,a
2,9,Y,b
3,4,B,b
4,-1,C,c


In [31]:
df_issues, df_issue = ch_checks_imported.table_look(dict_data['df_checks_issues.pkl'], 0)
ICD.display(df_issues)
ICD.display(df_issue)

,key_1,key_2,key_3,file,sub_file,step_number,category,issue_short_desc,issue_long_desc,column,issue_count,issue_idx,grouping
0,4,None,None,df_checks_issues.pkl,NaN,0,NaN,Number should be greater than 0,,NaN,1,4,2019-11-24 14:12:44.269445


,number,category_1,category_2
4,-1,C,c


# 5. Formed tables

## Initialise the class

In [32]:
var_start_time = datetime.now()
dc_summary = DataCuration(var_start_time, '5')

## Set the data

In [33]:
dc_summary.set_table(pickle.load(open('data/df_summary.pkl', 'rb')))

## Create summaries

In [34]:
def func_value_counts(df, col):
    s_out = df[col].value_counts()
    df_out = s_out.to_frame('count')
    df_out.reset_index(inplace=True)
    df_out.rename(columns={'index': col}, inplace=True)
    
    return df_out


def func_form_tables(tables, formed_tables, grouping, key_1, key_2, key_3, key_separator, **kwargs):
    dict_summaries = dict()
    if type(tables).__name__ == 'DataFrame':
        dict_summaries['original_table'] = tables
    dict_summaries['value counts on `str`'] = func_value_counts(tables, 'str')
    dict_summaries['value counts on `str_2`'] = func_value_counts(tables, 'str_2')
    
    return dict_summaries

In [35]:
dc_summary.form_summary_tables(function=func_form_tables)

In [36]:
for key in dc_summary.formed_tables.keys():
    print(key)
    ICD.display(dc_summary.formed_tables[key])

original_table


,str,str_2,count,int_max
0,b,c,1,6
1,d,b,1,9
2,c,b,1,0
3,d,d,1,9
4,c,b,1,1
5,a,d,1,3
...,...,...,...,...
44,d,b,1,8
45,d,b,1,10
46,d,e,1,10


value counts on `str`


,str,count
0,a,17
1,d,14
2,c,10
3,b,9


value counts on `str_2`


,str_2,count
0,c,16
1,e,14
2,b,11
3,d,9


---
**GigiSR**